In [ ]:
import numpy as np
import json
from PIL import Image, ImageDraw
import os
import cv2
import pandas as pd
from tqdm import tqdm
import shutil
import random

from face_detector import FaceDetector
# this face detector is taken from here
# https://github.com/TropComplique/FaceBoxes-tensorflow
# (facial keypoints detector will be trained to work well with this detector)

The purpose of this script is to explore images/annotations of the CelebA dataset.  
Also it cleans CelebA.  
Also it converts annotations into json format.

In [ ]:
IMAGES_DIR = '/home/dan/datasets/img_celeba.7z/out/'
ANNOTATIONS_PATH = '/home/gpu2/hdd/dan/CelebA/list_landmarks_celeba.txt'
SPLIT_PATH = '/home/gpu2/hdd/dan/CelebA/list_eval_partition.txt'

# Read data

In [ ]:
# collect paths to all images

all_paths = []
for name in tqdm(os.listdir(IMAGES_DIR)):
    all_paths.append(os.path.join(IMAGES_DIR, name))

metadata = pd.DataFrame(all_paths, columns=['full_path'])

# strip root folder
metadata['name'] = metadata.full_path.apply(lambda x: os.path.relpath(x, IMAGES_DIR))

In [ ]:
# number of images is taken from the official website
assert len(metadata) == 202599

In [ ]:
# see all unique endings
metadata.name.apply(lambda x: x.split('.')[-1]).unique()

### Detect a face on each image

In [ ]:
face_detector = FaceDetector('model-step-240000.pb', visible_device_list='0')

In [ ]:
detections = []
for p in tqdm(metadata.full_path):
    image = cv2.imread(p)
    image = image[:, :, [2, 1, 0]]  # to RGB
    detections.append(face_detector(image))

In [ ]:
# take only images where one high confidence box is detected
bad_images = [metadata.name[i] for i, (b, s) in enumerate(detections) if len(b) != 1 or s.max() < 0.5]

In [ ]:
boxes = {}
for n, (box, score) in zip(metadata.name, detections):
    if n not in bad_images:
        ymin, xmin, ymax, xmax = box[0]
        boxes[n] = (xmin, ymin, xmax, ymax)

In [ ]:
# read keypoints

def get_numbers(s):
    s = s.strip().split(' ')
    return [s[0]] + [int(i) for i in s[1:] if i]
    
with open(ANNOTATIONS_PATH, 'r') as f:
    content = f.readlines()
    content = content[2:]
    content = [get_numbers(s) for s in content]

landmarks = {}
more_bad_images = []
for i in content:
    name = i[0]
    
    keypoints = [
        [i[1], i[2]],  # lefteye_x lefteye_y 
        [i[3], i[4]],  # righteye_x righteye_y
        [i[5], i[6]],  # nose_x nose_y 
        [i[7], i[8]],  # leftmouth_x leftmouth_y
        [i[9], i[10]],  # rightmouth_x rightmouth_y
    ]
    
    # assert that landmarks are inside the box
    if name in bad_images:
        continue
    xmin, ymin, xmax, ymax = boxes[name]
    points = np.array(keypoints)
    is_normal = (points[:, 0] > xmin).all() and\
        (points[:, 0] < xmax).all() and\
        (points[:, 1] > ymin).all() and\
        (points[:, 1] < ymax).all()
    if not is_normal:
        more_bad_images.append(name)

    landmarks[name] = keypoints 

In [ ]:
# number of weird landmarks
len(more_bad_images)

In [ ]:
to_remove = more_bad_images + bad_images
metadata = metadata.loc[~metadata.name.isin(to_remove)]
metadata = metadata.reset_index(drop=True)

In [ ]:
# size after cleaning
len(metadata)

# Show some bounding boxes and landmarks

In [ ]:
def draw_boxes_on_image(path, box, keypoints):

    image = Image.open(path)
    draw = ImageDraw.Draw(image, 'RGBA')

    xmin, ymin, xmax, ymax = box
    fill = (255, 255, 255, 45)
    outline = 'red'
    draw.rectangle(
        [(xmin, ymin), (xmax, ymax)],
        fill=fill, outline=outline
    )
    
    for x, y in keypoints:
        draw.ellipse([
            (x - 2.0, y - 2.0),
            (x + 2.0, y + 2.0)
        ], outline='red')

    return image

In [ ]:
i = random.randint(0, len(metadata) - 1)  # choose a random image
some_boxes = boxes[metadata.name[i]]
keypoints = landmarks[metadata.name[i]]
draw_boxes_on_image(metadata.full_path[i], some_boxes, keypoints)

# Create train-val split

In [ ]:
split = pd.read_csv(SPLIT_PATH, header=None, sep=' ')
split.columns = ['name', 'assignment']

In [ ]:
split = split.loc[~split.name.isin(to_remove)]
split = split.reset_index(drop=True)

In [ ]:
split.assignment.value_counts()

In [ ]:
# "0" represents training image, "1" represents validation image, "2" represents testing image
train = list(split.loc[split.assignment.isin([0, 1]), 'name'])
val = list(split.loc[split.assignment.isin([2]), 'name'])

# Convert

In [ ]:
def get_annotation(name, width, height):
    xmin, ymin, xmax, ymax = boxes[name]
    keypoints = landmarks[name]
    annotation = {
        "filename": name,
        "size": {"depth": 3, "width": width, "height": height},
        "box": {"ymin": int(ymin), "ymax": int(ymax), "xmax": int(xmax), "xmin": int(xmin)},
        "landmarks": keypoints
    }
    return annotation

In [ ]:
# create folders for the converted dataset
TRAIN_DIR = '/home/dan/datasets/CelebA/train/'
shutil.rmtree(TRAIN_DIR, ignore_errors=True)
os.mkdir(TRAIN_DIR)
os.mkdir(os.path.join(TRAIN_DIR, 'images'))
os.mkdir(os.path.join(TRAIN_DIR, 'annotations'))

VAL_DIR = '/home/dan/datasets/CelebA/val/'
shutil.rmtree(VAL_DIR, ignore_errors=True)
os.mkdir(VAL_DIR)
os.mkdir(os.path.join(VAL_DIR, 'images'))
os.mkdir(os.path.join(VAL_DIR, 'annotations'))

In [ ]:
for T in tqdm(metadata.itertuples()):
    
    # get width and height of an image
    image = cv2.imread(T.full_path)
    h, w, c = image.shape
    assert c == 3
    
    # name of the image
    name = T.name
    assert name.endswith('.jpg')
    
    if name in train:
        result_dir = TRAIN_DIR
    elif name in val:
        result_dir = VAL_DIR
    else:
        print('WTF')
        break

    # copy the image
    shutil.copy(T.full_path, os.path.join(result_dir, 'images', name))
    
    # save annotation for it
    d = get_annotation(name, w, h)
    json_name = name[:-4] + '.json'
    json.dump(d, open(os.path.join(result_dir, 'annotations', json_name), 'w')) 